<center><h1 style="color:#D55250;">Breast Cancer Classification🧬</h1></center>
<p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
    <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>

    


<center><img src="https://image.flaticon.com/icons/png/512/2659/2659980.png" width=500></center>
        <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
            <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
        
        
<center><h2 style="color:#E77A80;">What is the Cancer?</h2></center>
        <center><h4 style="color:#4E5130;">Cancer is the name given to all associated diseases. In all types of cancer, some of the body cells begin to divide and spread to surrounding tissues. Cancer can start almost anywhere in the human body, made up of trillions of cells. Normally, human cells grow and multiply to form new cells when the body needs it. When cells become old or damaged, they die and new cells replace the dying cells.</h4></center>
                <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
                
<center><h2 style="color:#E77A70;">Cancer Treatment</h2></center>
                
<center><h4 style="color:#4E5130;">Common treatment methods used in cancer are surgery, radiotherapy and chemotherapy. Less frequently, hormone therapies, biological therapies, and targeted therapies are used. These treatment methods are applied alone or together. The first treatment is generally known as first line treatment. Treatment given after first-line therapy is called adjuvant therapy. Chemotherapy applied after surgical treatment is an adjuvant treatment. Neoadjuvant therapy is the treatment applied before the first step treatment.</h4></center>
                    <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
                    
                    
<center><h2 style="color:#E77A80;">Content</h2></center>

[<center>Libarys📚</center>](#1)

[<center>Load Dataset👨‍🔬</center>](#2)

[<center>Cleaning Data And EDA🧹</center>](#3)
                        
[<center>Outliers🧺</center>](#4)

[<center>Standardization And Seperation🔨</center>](#5)

[<center>Classification With K-Nearest Neighbors🌌</center>](#6)
                        
[<center>K-Nearest Neighbors Optimization🏋</center>](#7)
                        
[<center>Principal Component Analysis🔬</center>](#8)
                        
[<center>NCA🚀</center>](#9)
                        
[<center>Status Summary📌</center>](#10)
                        
<p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
<p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
                                
                                
                        
<center><h2 style="color:#D55250;">Important Note❗️</h2></center>

<center><h4 style="color:#4E5130;">I posted all the processes I have done on the notebook called What is machine learning, you can access it in more detail from there.</h4></center>
<center><h4 style="color:#4E5130;">That's why I did not explain much while trading here. Good reading</h4></center>

[<center>⚙️Machine Learning Tutorial For Beginners</center>](https://www.kaggle.com/omercansvgn/machine-learning-tutorial-for-beginners/)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<a id="1"></a> <br>
<center><h1 style="color:#D55250;">Libarys📚</h1></center>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier,NeighborhoodComponentsAnalysis,LocalOutlierFactor
from sklearn.decomposition import PCA
from warnings import filterwarnings
filterwarnings('ignore')

<a id="2"></a> <br>
<center><h1 style="color:#D55250;">Load Dataset👨‍🔬</h1></center>

In [ ]:
data = pd.read_csv("../input/breast-cancer-wisconsin-data/data.csv")
data.head()

<a id="3"></a> <br>
<center><h1 style="color:#D55250;">Cleaning Data And EDA🧹</h1></center>

<center><h4 style="color:#4E5130;">Missing observations here and I will fix a few problems</h4></center>

In [ ]:
data = data.rename(columns= {'diagnosis':'target'})#I want to change the diagnosis variable name to target.
data.drop(["Unnamed: 32","id"],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
data['target'] = [1 if i.strip() == 'M'else 0 for i in data.target] # I need to convert the M and B in the target variable to 0 and 1.
# .strip () removes spaces in string expressions.

In [ ]:
# How many M and how many B's are we examining them.
palette=["#FBC00E","#29B3FF"]
sns.countplot(data['target'],palette=palette);
print(data.target.value_counts())

In [ ]:
# 1 malignant so M
# 0 benign so B
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()
# strictly standardization process is required for this data

In [ ]:
# Since all variables we have are numeric, we look at corr ().
ax = plt.figure(figsize=(15,8))
ax = sns.heatmap(data.corr());

In [ ]:
# Since this is so confusing, I'll set a Threshold and cover those above it.
cor_mat = data.corr()
threshold = 0.75
filters = np.abs(cor_mat['target']) > threshold
corr_features = cor_mat.columns[filters].tolist()
ax = plt.figure(figsize=(15,8))
ax = sns.heatmap(data[corr_features].corr(),annot=True,linewidths=.3)
plt.title('Correlation Between Features w Corr Threshold 0.75');

In [ ]:
data_melted = pd.melt(data,id_vars='target',var_name='features',value_name='value')
ax = plt.figure(figsize=(15,8))
ax = sns.boxplot(x='features',y='value',hue='target',data=data_melted,palette=palette)
plt.xticks(rotation=90);
# Because the data is not standardized here, it becomes a strange table, we will use it later.

In [ ]:
# Pair plot is one of the most effective methods used in numerical data.
# This will not look nice either, because the data needs to be standardized.
sns.pairplot(data[corr_features],diag_kind='kde',markers='+',hue='target',palette=palette);
# But I just used corr_features for images.

<center><h2 style="color:#4E5130;">There is a distortion, we need to fix it.</h2></center>

<a id="4"></a> <br>
<center><h1 style="color:#D55250;">Outliers🧺</h1></center>

<center><h4 style="color:#4E5130;">Here we will examine outlier observations and deal with them.</h4></center>

In [ ]:
y = data['target']
x = data.drop(['target'],axis=1)
columns = x.columns.tolist()

In [ ]:
clf = LocalOutlierFactor()
y_pred = clf.fit_predict(x)
x_score = clf.negative_outlier_factor_
outlier_score = pd.DataFrame()
outlier_score['score'] = x_score
outlier_score.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(x.iloc[:,0],x.iloc[:,1],color='#29B3FF',s=3,label='Data Points')
plt.legend();

In [ ]:
radius = (x_score.max() - x_score) / (x_score.max() - x_score.min())

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(x.iloc[:,0],x.iloc[:,1],color='#FBC00E',s=3,label='Data Points')
plt.scatter(x.iloc[:,0],x.iloc[:,1],s=1000*radius,edgecolors='#29B3FF',facecolors='none',label='Outlier Scores')
plt.legend()
plt.show()

In [ ]:
# We are looking at contradictory observations.
threshold = -2.5
filtre = outlier_score['score'] < threshold
outlier_index = outlier_score[filtre].index.tolist()
plt.figure(figsize=(8,5))
plt.scatter(x.iloc[outlier_index,0],x.iloc[outlier_index,1],color='#D55250',s=50,label='Outlier')
plt.scatter(x.iloc[:,0],x.iloc[:,1],color='#FBC00E',s=3,label='Data Points')
plt.scatter(x.iloc[:,0],x.iloc[:,1],s=1000*radius,edgecolors='#29B3FF',facecolors='none',label='Outlier Scores')
plt.legend()
plt.show()

In [ ]:
# Drop outliers
x = x.drop(outlier_index)
y = y.drop(outlier_index).values
# All of these, there are a lot of other variables waiting for outlier observations for columns 0 and 1.

<a id="5"></a> <br>
<center><h1 style="color:#D55250;">Standardization And Seperation🔨</h1></center>

<center><h4 style="color:#4E5130;">Our data consists of very different numbers, which affects our correct classification rate, so we need to standardize the data.</h4></center>

In [ ]:
# Train Test seperation
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [ ]:
# Standart
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
# Let's visualize boxplot that was not visualized before.
x_train_df = pd.DataFrame(x_train,columns=columns)
x_train_df['target'] = y_train
data_melted = pd.melt(x_train_df,id_vars='target',var_name='features',value_name='value')
plt.figure(figsize=(15,8))
sns.boxplot(x='features',y='value',hue='target',data=data_melted,palette=palette)
plt.xticks(rotation=90)
plt.show()

<a id="6"></a> <br>
<center><h1 style="color:#D55250;">Classification With K-Nearest Neighbors🌌</h1></center>

<center><h4 style="color:#4E5130;">In this section I will classify the data set we cleared.</h4></center>

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
cm = confusion_matrix(y_test,y_pred)
acc = accuracy_score(y_test,y_pred)
score = knn.score(x_test,y_test)
print('Score:',score)
print('Confusion Matrix:',cm)
print('Basic Accuracy Score:',acc)

<a id="7"></a> <br>
<center><h1 style="color:#D55250;">K-Nearest Neighbors Optimization🏋🏻</h1></center>

<center><h4 style="color:#4E5130;">We will perform the hyperparameter optimization process of our algorithm.</h4></center>

In [ ]:
def knn_best_params(x_train,x_test,y_train,y_test):
    k_range = list(range(1,31))
    weight_options = ['uniform','distance']
    print()
    param_grid = dict(n_neighbors = k_range,weights=weight_options)
    knn = KNeighborsClassifier()
    grid = GridSearchCV(knn,param_grid,cv=10,scoring='accuracy')
    grid.fit(x_train,y_train)
    print('Best Training Score {} with parameters: {}'.format(grid.best_score_,grid.best_params_))
    print()
    
    knn = KNeighborsClassifier(**grid.best_params_)
    knn.fit(x_train,y_train)
    
    y_pred_test = knn.predict(x_test)
    y_pred_train = knn.predict(x_train)
    cm_test = confusion_matrix(y_test,y_pred_test)
    cm_train = confusion_matrix(y_train,y_pred_train)
    acc_test = accuracy_score(y_test,y_pred_test)
    acc_train = accuracy_score(y_train,y_pred_train)
    print('Test Score: {},Train Score: {}'.format(acc_test,acc_train))
    print()
    print('Confusion Matrix Test: {}'.format(cm_test))
    print('Confusion Matrix Train: {}'.format(cm_train))
    
    return grid

In [ ]:
grid = knn_best_params(x_train,x_test,y_train,y_test)

<a id="8"></a> <br>
<center><h1 style="color:#D55250;">Principal Component Analysis🔬</h1></center>

<center><h4 style="color:#4E5130;">We will try to explain our results with fewer variables.</h4></center>

In [ ]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
pca = PCA(n_components=2)
pca.fit(x_scaled)
x_reduced_pca = pca.transform(x_scaled)
pca_data = pd.DataFrame(x_reduced_pca,columns=['p1','p2'])
pca_data['target'] = y
plt.figure(figsize=(8,5))
sns.scatterplot(x='p1',y='p2',hue='target',data=pca_data,palette=palette)
plt.title('PCA: p1 vs p2')
plt.show()
# We reduced 30 dimensional data to 2 dimensions with PCA.

In [ ]:
# Now we will do a chnn using 2 dimensional data.
x_train_pca,x_test_pca,y_train_pca,y_test_pca = train_test_split(x_reduced_pca,y,test_size=0.3,random_state=42)
grid_pca = knn_best_params(x_train_pca,x_test_pca,y_train_pca,y_test_pca)

In [ ]:
# We use a visualization to see how the split is decided.
cmap_light = ListedColormap(['#FBC00E','#29B3FF'])
cmap_bold = ListedColormap(['darkorange','darkblue'])
h = .05
X = x_reduced_pca
x_min,x_max = X[:,0].min() -1,X[:,0].max() + 1
y_min,y_max = X[:,1].min() -1 ,X[:,1].max() + 1
xx,yy = np.meshgrid(np.arange(x_min,x_max,h),
                   np.arange(y_min,y_max,h))
Z = grid_pca.predict(np.c_[xx.ravel(),yy.ravel()])
Z = Z.reshape(xx.shape)
plt.figure(figsize=(10,8))
plt.pcolormesh(xx,yy,Z,cmap=cmap_light)
plt.scatter(X[:,0],X[:,1],c=y,cmap=cmap_bold,
           edgecolors='k',s=20)
plt.xlim(xx.min(),xx.max())
plt.ylim(yy.min(),yy.max())
plt.title("%i-Class Classification (k= %i,weights = '%s')"%(len(np.unique(y)),grid_pca.best_estimator_.n_neighbors,grid_pca.best_estimator_.weights))

<a id="9"></a> <br>
<center><h1 style="color:#D55250;">NCA🚀</h1></center>

In [ ]:
nca = NeighborhoodComponentsAnalysis(n_components=2,random_state=42)
nca.fit(x_scaled,y)
x_reduced_nca = nca.transform(x_scaled)
nca_data = pd.DataFrame(x_reduced_nca,columns=['p1','p2'])
nca_data['target'] = y
plt.figure(figsize=(10,8))
sns.scatterplot(x='p1',y='p2',hue='target',data=nca_data,palette=palette)
plt.title('NCA : p1 vs p2')
plt.show()

In [ ]:
x_train_nca,x_test_nca,y_train_nca,y_test_nca = train_test_split(x_reduced_nca,y,test_size=0.3,random_state=42)
grid_nca = knn_best_params(x_train_nca,x_test_nca,y_train_nca,y_test_nca)

<a id="10"></a> <br>
<center><h1 style="color:#D55250;">Status Summary📌</h1></center>

<center><h4 style="color:#4E5130;">Yes we got a very high score as you can see.In real life problems these methods and algorithms work more or less like this.Here we just covered a problem for KNN, but I showed you a lot of algorithms, you can use them all.You just need to understand the problem correctly and make the right decisions.I did not explain too much while addressing the problem because I explained everything in detail in the tutorial.I hope this real life problem works for you too</h4></center>

<center><h3 style="color:#E77A80;">Thank you for reading, don't forget to give upvote if you find it really useful, I wish you a pleasant day</h3></center>

<center><img src="https://images.vexels.com/media/users/3/161144/isolated/preview/8a71d12a40321402455aa974b63c44c1-thank-you-thanks-badge-sticker-by-vexels.png" width=250></center>
